# Fine-Tuning Model Parameters via a Randomized Search

In [23]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal
import numpy as np

## Get the data

In [2]:
housing = fetch_california_housing()

### Create train, validation and test sets

In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [5]:
X_train.shape

(11610, 8)

In [6]:
X_test.shape

(5160, 8)

In [7]:
X_valid.shape

(3870, 8)

In [8]:
y_train.shape

(11610,)

In [9]:
y_test.shape

(5160,)

In [10]:
y_valid.shape

(3870,)

### Scale the data

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [12]:
X_train.shape[1:]

(8,)

## Create the model

In [38]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [19]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [20]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

## Specify the randomized search parameters

In [24]:
param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}

## Train the model by iterating over the randomized parameter search

In [25]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(
    X_train,
    y_train,
    epochs=20,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Epoch 1/20
242/242 [==============================] - ETA: 0s - loss: 3.1694 - 0s 964us/step - loss: 2.8008 - val_loss: 0.5407
Epoch 2/20
242/242 [==============================] - 0s 545us/step - loss: 4.1743 - val_loss: 1.1278
Epoch 3/20
242/242 [==============================] - 0s 546us/step - loss: 22.0868 - val_loss: 3.0870
Epoch 4/20
242/242 [==============================] - 0s 534us/step - loss: 79.5358 - val_loss: 24.0339
Epoch 5/20
242/242 [==============================] - 0s 517us/step - loss: 93.1564 - val_loss: 230.9524
Epoch 6/20
242/242 [==============================] - 0s 526us/step - loss: 16568.6797 - val_loss: 955.9703
Epoch 7/20
242/242 [==============================] - 0s 542us/step - loss: 65031.1608 - val_loss: 6689.5122
Epoch 8/20
242/242 [==============================] - 0s 526us/step - loss: 183939.6986 - val_loss: 52198.2070
Epoch 9/20
242/242 [==============================] - 0s 521us/step - loss: 4251488.3687 - val_loss: 327414.4062
Epoch 10/20
242/24

242/242 [==============================] - 0s 555us/step - loss: 0.5028 - val_loss: 0.4732
Epoch 7/20
242/242 [==============================] - 0s 637us/step - loss: 0.4668 - val_loss: 0.4663
Epoch 8/20
242/242 [==============================] - 0s 567us/step - loss: 0.4524 - val_loss: 0.4618
Epoch 9/20
242/242 [==============================] - 0s 546us/step - loss: 0.4428 - val_loss: 0.4591
Epoch 10/20
242/242 [==============================] - 0s 546us/step - loss: 0.4217 - val_loss: 0.4595
Epoch 11/20
242/242 [==============================] - 0s 512us/step - loss: 0.4384 - val_loss: 0.4484
Epoch 12/20
242/242 [==============================] - 0s 619us/step - loss: 0.4079 - val_loss: 0.4476
Epoch 13/20
242/242 [==============================] - 0s 487us/step - loss: 0.4037 - val_loss: 0.4396
Epoch 14/20
242/242 [==============================] - 0s 550us/step - loss: 0.4173 - val_loss: 0.4358
Epoch 15/20
242/242 [==============================] - 0s 488us/step - loss: 0.4123 - va

242/242 [==============================] - 0s 516us/step - loss: 0.7756 - val_loss: 0.7038
Epoch 4/20
242/242 [==============================] - 0s 517us/step - loss: 0.6621 - val_loss: 0.6467
Epoch 5/20
242/242 [==============================] - 0s 523us/step - loss: 0.5991 - val_loss: 0.6248
Epoch 6/20
242/242 [==============================] - 0s 504us/step - loss: 0.5886 - val_loss: 0.6126
Epoch 7/20
242/242 [==============================] - 0s 529us/step - loss: 0.5845 - val_loss: 0.6030
Epoch 8/20
242/242 [==============================] - 0s 509us/step - loss: 0.5642 - val_loss: 0.5942
Epoch 9/20
242/242 [==============================] - 0s 507us/step - loss: 0.5424 - val_loss: 0.5872
Epoch 10/20
242/242 [==============================] - 0s 503us/step - loss: 0.5525 - val_loss: 0.5812
Epoch 11/20
242/242 [==============================] - 0s 517us/step - loss: 0.5380 - val_loss: 0.5758
Epoch 12/20
242/242 [==============================] - 0s 502us/step - loss: 0.5514 - val_l

121/121 [==============================] - 0s 314us/step - loss: 0.3967
Epoch 1/20
242/242 [==============================] - 0s 1ms/step - loss: 4.2189 - val_loss: 0.9026
Epoch 2/20
242/242 [==============================] - 0s 610us/step - loss: 0.7267 - val_loss: 0.5724
Epoch 3/20
242/242 [==============================] - 0s 584us/step - loss: 0.5084 - val_loss: 0.5451
Epoch 4/20
242/242 [==============================] - 0s 563us/step - loss: 0.5387 - val_loss: 0.5404
Epoch 5/20
242/242 [==============================] - 0s 535us/step - loss: 0.5175 - val_loss: 0.5393
Epoch 6/20
242/242 [==============================] - 0s 537us/step - loss: 0.5303 - val_loss: 0.5384
Epoch 7/20
242/242 [==============================] - 0s 477us/step - loss: 0.4963 - val_loss: 0.5395
Epoch 8/20
242/242 [==============================] - 0s 484us/step - loss: 0.5212 - val_loss: 0.5364
Epoch 9/20
242/242 [==============================] - 0s 511us/step - loss: 0.5091 - val_loss: 0.5381
Epoch 10/20


242/242 [==============================] - 0s 504us/step - loss: 4220450.8954 - val_loss: 1137863.3750
Epoch 5/20
242/242 [==============================] - 0s 510us/step - loss: 545284554.2828 - val_loss: 146647136.0000
Epoch 6/20
242/242 [==============================] - 0s 505us/step - loss: 22620741029.2510 - val_loss: 70243672064.0000
Epoch 7/20
242/242 [==============================] - 0s 510us/step - loss: 573149820692.0165 - val_loss: 6716836020224.0000
Epoch 8/20
242/242 [==============================] - 0s 505us/step - loss: 3595819715815230.0000 - val_loss: 1062652822421504.0000
Epoch 9/20
242/242 [==============================] - 0s 504us/step - loss: 74500880852771184.0000 - val_loss: 162651425111801856.0000
Epoch 10/20
242/242 [==============================] - 0s 499us/step - loss: 135816283122378899456.0000 - val_loss: 130005419239397130240.0000
Epoch 11/20
121/121 [==============================] - 0s 290us/step - loss: 2385697178631530348544.0000
Epoch 1/20
242/24

Epoch 17/20
242/242 [==============================] - 0s 525us/step - loss: 0.4061 - val_loss: 0.4582
Epoch 18/20
242/242 [==============================] - 0s 550us/step - loss: 0.4001 - val_loss: 0.4554
Epoch 19/20
242/242 [==============================] - 0s 531us/step - loss: 0.4018 - val_loss: 0.5202
Epoch 20/20
121/121 [==============================] - 0s 287us/step - loss: 0.4666
Epoch 1/20
242/242 [==============================] - 0s 814us/step - loss: 2.2581 - val_loss: 0.7527
Epoch 2/20
242/242 [==============================] - 0s 525us/step - loss: 0.7126 - val_loss: 6.3886
Epoch 3/20
242/242 [==============================] - 0s 505us/step - loss: 33.8813 - val_loss: 236.5398
Epoch 4/20
242/242 [==============================] - 0s 507us/step - loss: 50921.0318 - val_loss: 12254.3760
Epoch 5/20
242/242 [==============================] - 0s 515us/step - loss: 1299770.7080 - val_loss: 683270.8750
Epoch 6/20
242/242 [==============================] - 0s 508us/step - loss:

242/242 [==============================] - 0s 458us/step - loss: 0.9454 - val_loss: 0.9335
Epoch 11/20
242/242 [==============================] - 0s 521us/step - loss: 0.8337 - val_loss: 0.8617
Epoch 12/20
242/242 [==============================] - 0s 531us/step - loss: 0.8164 - val_loss: 0.8083
Epoch 13/20
242/242 [==============================] - 0s 531us/step - loss: 0.6975 - val_loss: 0.7685
Epoch 14/20
242/242 [==============================] - 0s 528us/step - loss: 0.7366 - val_loss: 0.7386
Epoch 15/20
242/242 [==============================] - 0s 525us/step - loss: 0.6480 - val_loss: 0.7160
Epoch 16/20
242/242 [==============================] - 0s 512us/step - loss: 0.6616 - val_loss: 0.6987
Epoch 17/20
242/242 [==============================] - 0s 526us/step - loss: 0.6569 - val_loss: 0.6852
Epoch 18/20
242/242 [==============================] - 0s 543us/step - loss: 0.6382 - val_loss: 0.6748
Epoch 19/20
242/242 [==============================] - 0s 527us/step - loss: 0.6229 -

242/242 [==============================] - 0s 517us/step - loss: 0.4951 - val_loss: 0.7153
Epoch 16/20
242/242 [==============================] - 0s 523us/step - loss: 0.5461 - val_loss: 0.5356
Epoch 17/20
242/242 [==============================] - 0s 508us/step - loss: 0.4796 - val_loss: 0.6892
Epoch 18/20
242/242 [==============================] - 0s 514us/step - loss: 0.5503 - val_loss: 0.5354
Epoch 19/20
242/242 [==============================] - 0s 510us/step - loss: 0.4886 - val_loss: 0.6902
Epoch 20/20
121/121 [==============================] - 0s 291us/step - loss: 0.5415


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000021FB8BC20A0>, as the constructor either does not set or modifies parameter learning_rate

## Get the parameter set that performed best and its score

In [28]:
rnd_search_cv.best_params_

{'learning_rate': 0.005679089392533315, 'n_hidden': 2, 'n_neurons': 73}

In [35]:
rnd_search_cv.best_score_

-0.37595300873120624

## Retrain the model with the best parameters

In [41]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(
    build_model,
    learning_rate=rnd_search_cv.best_params_['learning_rate'],
    n_hidden=rnd_search_cv.best_params_['n_hidden'],
    n_neurons=rnd_search_cv.best_params_['n_neurons']
)
keras_reg.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Epoch 1/100
363/363 [==============================] - 0s 760us/step - loss: 1.5837 - root_mean_squared_error: 1.2339 - val_loss: 0.5966 - val_root_mean_squared_error: 0.7724
Epoch 2/100
363/363 [==============================] - 0s 532us/step - loss: 0.5232 - root_mean_squared_error: 0.7231 - val_loss: 0.5169 - val_root_mean_squared_error: 0.7189
Epoch 3/100
363/363 [==============================] - 0s 539us/step - loss: 0.4933 - root_mean_squared_error: 0.7022 - val_loss: 0.6156 - val_root_mean_squared_error: 0.7846
Epoch 4/100
363/363 [==============================] - 0s 548us/step - loss: 0.4570 - root_mean_squared_error: 0.6760 - val_loss: 0.4642 - val_root_mean_squared_error: 0.6813
Epoch 5/100
363/363 [==============================] - 0s 534us/step - loss: 0.4348 - root_mean_squared_error: 0.6592 - val_loss: 0.4486 - val_root_mean_squared_error: 0.6698
Epoch 6/100
363/363 [==============================] - 0s 539us/step - loss: 0.4126 - root_mean_squared_error: 0.6422 - val_l

363/363 [==============================] - 0s 504us/step - loss: 0.2848 - root_mean_squared_error: 0.5333 - val_loss: 0.3284 - val_root_mean_squared_error: 0.5731
Epoch 48/100
363/363 [==============================] - 0s 539us/step - loss: 0.2973 - root_mean_squared_error: 0.5452 - val_loss: 0.3255 - val_root_mean_squared_error: 0.5705
Epoch 49/100
363/363 [==============================] - 0s 547us/step - loss: 0.3106 - root_mean_squared_error: 0.5572 - val_loss: 0.3348 - val_root_mean_squared_error: 0.5786
Epoch 50/100
363/363 [==============================] - 0s 540us/step - loss: 0.3077 - root_mean_squared_error: 0.5544 - val_loss: 0.3230 - val_root_mean_squared_error: 0.5683
Epoch 51/100
363/363 [==============================] - 0s 548us/step - loss: 0.2776 - root_mean_squared_error: 0.5267 - val_loss: 0.3432 - val_root_mean_squared_error: 0.5858
Epoch 52/100
363/363 [==============================] - 0s 542us/step - loss: 0.2968 - root_mean_squared_error: 0.5445 - val_loss: 0.

363/363 [==============================] - 0s 543us/step - loss: 0.2604 - root_mean_squared_error: 0.5102 - val_loss: 0.3135 - val_root_mean_squared_error: 0.5599
Epoch 94/100
363/363 [==============================] - 0s 539us/step - loss: 0.2682 - root_mean_squared_error: 0.5178 - val_loss: 0.3129 - val_root_mean_squared_error: 0.5594
Epoch 95/100
363/363 [==============================] - 0s 546us/step - loss: 0.2777 - root_mean_squared_error: 0.5268 - val_loss: 0.3033 - val_root_mean_squared_error: 0.5507
Epoch 96/100
363/363 [==============================] - 0s 557us/step - loss: 0.2644 - root_mean_squared_error: 0.5140 - val_loss: 0.3053 - val_root_mean_squared_error: 0.5525
Epoch 97/100
363/363 [==============================] - 0s 546us/step - loss: 0.2561 - root_mean_squared_error: 0.5056 - val_loss: 0.3082 - val_root_mean_squared_error: 0.5552
Epoch 98/100
363/363 [==============================] - 0s 535us/step - loss: 0.2579 - root_mean_squared_error: 0.5077 - val_loss: 0.